In [ ]:
%matplotlib inline

In [ ]:
import os
import time
import xarray as xr
import numpy as np
from IPython import display

import redis

import matplotlib.pyplot as plt
# from bokeh.io import output_notebook
# output_notebook()



In [ ]:
r = redis.StrictRedis(host='127.0.0.1', port=6379, db=0, password=os.getenv('REDIS_PASSWORD'))

In [ ]:
def get():
    key, vals = r.brpop('A')
    data = np.fromstring(vals, dtype='<i4').reshape((100, 100), order='F')
    return xr.DataArray(data,  dims=('x', 'y'), name=key)


# def get():
#     '''for testing while server is down'''
#     return xr.DataArray(np.random.randint(-1, 1, size=(100, 100)), dims=('x', 'y'), name=key)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4), sharex=True, sharey=True, )
fig.tight_layout(w_pad=3.0)
old = get()

old.plot.pcolormesh(ax=axes[0], vmin=-1, vmax=1, cmap='viridis')
old.plot.pcolormesh(ax=axes[1], vmin=-1, vmax=1, cmap='viridis')
old.plot.pcolormesh(ax=axes[2], vmin=-1, vmax=1, cmap='RdBu')

counter = 1
while True:
    new = get()
    old.plot.pcolormesh(ax=axes[0], vmin=-1, vmax=1, cmap='viridis', add_colorbar=False)
    new.plot.pcolormesh(ax=axes[1], vmin=-1, vmax=1, cmap='viridis', add_colorbar=False)
    (new - old).plot.pcolormesh(ax=axes[2], vmin=-1, vmax=1, cmap='RdBu', add_colorbar=False)
    axes[0].set_title('Old: %d' % counter)
    axes[1].set_title('New: %d' % (counter + 1))
    axes[2].set_title('Diff: %d' % (counter + 1))
    display.clear_output(wait=True)
    display.display(fig)
    time.sleep(5)
    old = new
    del new
    counter += 1

    